# Import Library

In [1]:
import numpy as np

In [2]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from keras.models import Model

Using TensorFlow backend.


In [4]:
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten

In [5]:
from keras.optimizers import Adam

In [6]:
from keras.utils.np_utils import to_categorical

In [7]:
from keras.callbacks import TensorBoard

In [8]:
from keras.datasets import fashion_mnist

# Data Preparation

In [9]:
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.

train_x = np.reshape(train_x, (len(train_x), 28, 28, 1))
test_x = np.reshape(test_x, (len(test_x), 28, 28, 1))

train_y = to_categorical( train_y )
test_y = to_categorical( test_y )

# Feature Extraction

In [10]:
inputs = Input(shape=(28, 28, 1))
conv_layer = ZeroPadding2D(padding=(2,2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1,1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1,1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1,1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(64, (3, 3), strides=(1,1), activation='relu')(conv_layer)

# Flatten Feature Map to Vector

In [11]:
flatten = Flatten()(conv_layer)

# Fully Connected Layer

In [12]:
fc_layer = Dense(256, activation='relu')(flatten)
fc_layer = Dense(64, activation='relu')(fc_layer)
outputs = Dense(10, activation='softmax')(fc_layer)

model = Model(inputs=inputs, outputs=outputs)

# Adam Optimizer dan Cross Entropy Loss

In [13]:
adam = Adam(lr=0.0001)

In [14]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary

In [15]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
__________

# Use TensorBoard

In [16]:
callbacks = TensorBoard(log_dir='./Graph')

# Train Data

In [17]:
model.fit(train_x, train_y, batch_size=256, epochs=5, verbose=1, validation_data=(test_x, test_y), callbacks=[callbacks])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 55s 923us/step - loss: 1.2728 - acc: 0.5833 - val_loss: 0.7775 - val_acc: 0.7068
Epoch 2/5
60000/60000 [==============================] - 48s 796us/step - loss: 0.6744 - acc: 0.7456 - val_loss: 0.6385 - val_acc: 0.7526
Epoch 3/5
60000/60000 [==============================] - 44s 726us/step - loss: 0.5841 - acc: 0.7793 - val_loss: 0.5684 - val_acc: 0.7834
Epoch 4/5
60000/60000 [==============================] - 44s 733us/step - loss: 0.5343 - acc: 0.8015 - val_loss: 0.5278 - val_acc: 0.7998
Epoch 5/5
60000/60000 [==============================] - 45s 757us/step - loss: 0.5023 - acc: 0.8153 - val_loss: 0.4960 - val_acc: 0.8151


# Save Weight

In [18]:
model.save_weights('weights.h5')